In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

In [15]:
embed1 = bert_embedding('barcode scanner')
embed_verb = bert_embedding('scan')
embed2 = bert_embedding('product')

In [16]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

In [17]:
records, summary, key = driver.execute_query("""
MATCH (n)-[r:RELATED_TO]->(m)
WITH n.name as subject, m.name as object, r.type as verb, n.app as app, n.feature as feature, vector.similarity.cosine(n.embedding, $embed_one) as subjectScore, 
    vector.similarity.cosine(m.embedding, $embed_two) as objectScore,
    vector.similarity.cosine(r.embedding, $embed_verb) as verbScore
RETURN subject, object, verb, app, feature, ((subjectScore * 0.5) + (objectScore * 0.3) + (verbScore * 0.2)) as score
ORDER BY score DESC LIMIT 10
""", embed_one=embed1, embed_two=embed2, embed_verb=embed_verb)

for row in records:
    print(row['subject'], row['verb'], row['object'], row['score'], row['app'])

inventory database has product 0.9966325044631958 POS Connect
discount tags are used for product 0.9955235838890076 Odoo POS
virtual web cart contains items 0.994790369272232 POS Connect
product information include pictures 0.9941840946674347 POS Connect
online inventory sync sync 0.9936092615127563 POS Connect
payment process is more efficient 0.9932812809944154 Odoo POS
Electronic shelf labels remove the need to print labels 0.9931538224220275 Odoo POS
downloadable windows service package handles communications 0.9920973062515259 POS Connect
product set allows products 0.9920451462268829 Odoo POS
Odoo Point of Sale allows Multi Employees per Session 0.9918312549591064 Odoo POS
